# RF (Baseline)

In [1]:
import time
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Train the model
start_time = time.time()
rf_model.fit(X_train, y_train)
train_time = time.time() - start_time

# Predict on test data
y_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {rf_accuracy * 100:.2f}%")
print(f"Training Time: {train_time:.2f} seconds")

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Random Forest Accuracy: 96.73%
Training Time: 15.17 seconds


# RF (Optuna)

In [ ]:
import time
import optuna
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(int)  # Ensure target labels are integers

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna optimization
def optuna_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(optuna_objective, n_trials=20)
best_params_optuna = study.best_params

# Train optimized RandomForest model using Optuna
rf_model_optuna = RandomForestClassifier(n_estimators=best_params_optuna['n_estimators'], max_depth=best_params_optuna['max_depth'], random_state=42, n_jobs=-1)
start_time = time.time()
rf_model_optuna.fit(X_train, y_train)
train_time_optuna = time.time() - start_time

y_pred_optuna = rf_model_optuna.predict(X_test)
rf_accuracy_optuna = accuracy_score(y_test, y_pred_optuna)

# Output results
print(f"Optuna Optimized Random Forest Accuracy: {rf_accuracy_optuna * 100:.2f}%")
print(f"Optuna Training Time: {train_time_optuna:.2f} seconds")
print(f"Optuna Best Parameters: n_estimators={best_params_optuna['n_estimators']}, max_depth={best_params_optuna['max_depth']}")

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
[I 2025-02-17 05:48:49,469] A new study created in memory with name: no-name-f66400ba-1992-409d-a5ca-648b316d13b3
[I 2025-02-17 05:49:06,638] Trial 0 finished with value: 0.9663571428571428 and parameters: {'n_estimators': 114, 'max_depth': 20}. Best is trial 0 with value: 0.9663571428571428.
[I 2025-02-17 05:49:35,213] Trial 1 finished with value: 0.9680714285714286 and parameters: {'n_estimators': 186, 'max_depth': 27}. Best is trial 1 with value: 0.9680714285714286.
[I 2025-02-17 05:49:47,077] Trial 2 finished with value: 0.9657142857

# RF (Bald Eagle)

In [ ]:
import numpy as np
import cupy as cp
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist
import time

# Load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape(len(x_train), -1) / 255.0, x_test.reshape(len(x_test), -1) / 255.0

# BEO Parameters
population_size = 10
generations = 5
explore_ratio = 0.5 
exploit_ratio = 0.5  

# Initialize population
population = [{'n_estimators': random.randint(50, 150), 'max_depth': random.randint(5, 30)} for _ in range(population_size)]

best_solution, best_score = None, float('-inf')

for gen in range(generations):
    print(f"Generation {gen + 1}/{generations} started...")
    start_time = time.time()
    scores = []
    for idx, params in enumerate(population):
        model = RandomForestClassifier(n_estimators=params['n_estimators'], max_depth=params['max_depth'], random_state=42, n_jobs=-1)
        model.fit(x_train, y_train)
        acc = accuracy_score(y_test, model.predict(x_test))
        scores.append((params, acc))
        print(f"  Particle {idx + 1}: n_estimators={params['n_estimators']}, max_depth={params['max_depth']}, Accuracy={acc:.4f}")
        
        if acc > best_score:
            best_score, best_solution = acc, params

    # BEO-inspired Adjustment (simple guided search)
    avg_n_estimators = np.mean([p['n_estimators'] for p, _ in scores])
    avg_max_depth = np.mean([p['max_depth'] for p, _ in scores])
    population = [{'n_estimators': int(avg_n_estimators + random.uniform(-20, 20)), 'max_depth': int(avg_max_depth + random.uniform(-5, 5))} for _ in range(population_size)]
    
    print(f"Generation {gen + 1} completed in {time.time() - start_time:.2f} seconds")
    print(f"Current Best Score: {best_score:.4f}")

print(f"\nFinal Best Hyperparameters: {best_solution}")
print(f"Final Best Accuracy: {best_score * 100:.2f}%")

# RF (Particle Swarm)

In [ ]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist
import time

# Load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape(len(x_train), -1) / 255.0, x_test.reshape(len(x_test), -1) / 255.0

# PSO Parameters
num_particles = 10  # Number of particles in the swarm
num_iterations = 5  # Number of iterations (generations)
w = 0.7  # Inertia weight
c1 = 1.5  # Cognitive coefficient
c2 = 1.5  # Social coefficient

# Initialize particles
particles = [{'n_estimators': random.randint(50, 200), 'max_depth': random.randint(5, 50)} for _ in range(num_particles)]
velocities = [{'n_estimators': random.uniform(-10, 10), 'max_depth': random.uniform(-5, 5)} for _ in range(num_particles)]
best_positions = particles.copy()
best_scores = [float('-inf')] * num_particles

# Global best
global_best_position = None
global_best_score = float('-inf')

for iteration in range(num_iterations):
    print(f"Iteration {iteration + 1}/{num_iterations} started...")
    start_time = time.time()
    
    for i, params in enumerate(particles):
        model = RandomForestClassifier(n_estimators=params['n_estimators'], max_depth=params['max_depth'], random_state=42, n_jobs=-1)
        model.fit(x_train, y_train)
        acc = accuracy_score(y_test, model.predict(x_test))
        
        if acc > best_scores[i]:
            best_scores[i] = acc
            best_positions[i] = params.copy()
        
        if acc > global_best_score:
            global_best_score = acc
            global_best_position = params.copy()
        
        print(f"  Particle {i + 1}: n_estimators={params['n_estimators']}, max_depth={params['max_depth']}, Accuracy={acc:.4f}")
    
    # Update velocities and positions
    for i in range(num_particles):
        velocities[i]['n_estimators'] = (w * velocities[i]['n_estimators'] +
                                         c1 * random.random() * (best_positions[i]['n_estimators'] - particles[i]['n_estimators']) +
                                         c2 * random.random() * (global_best_position['n_estimators'] - particles[i]['n_estimators']))
        velocities[i]['max_depth'] = (w * velocities[i]['max_depth'] +
                                      c1 * random.random() * (best_positions[i]['max_depth'] - particles[i]['max_depth']) +
                                      c2 * random.random() * (global_best_position['max_depth'] - particles[i]['max_depth']))
        
        particles[i]['n_estimators'] = max(10, int(particles[i]['n_estimators'] + velocities[i]['n_estimators']))
        particles[i]['max_depth'] = max(1, int(particles[i]['max_depth'] + velocities[i]['max_depth']))
    
    print(f"Iteration {iteration + 1} completed in {time.time() - start_time:.2f} seconds")
    print(f"Current Best Score: {global_best_score:.4f}")

print(f"\nFinal Best Hyperparameters: {global_best_position}")
print(f"Final Best Accuracy: {global_best_score * 100:.2f}%")

## SVM (Baseline)

In [ ]:
import time
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(int)  # Ensure target labels are integers

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train baseline SVM model
svm_model = SVC()
start_time = time.time()
svm_model.fit(X_train, y_train)
train_time_svm = time.time() - start_time

y_pred_svm = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)

# Output results
print(f"Baseline SVM Accuracy: {svm_accuracy * 100:.2f}%")
print(f"Baseline SVM Training Time: {train_time_svm:.2f} seconds")

## SVM (Optuna)

In [ ]:
import time
import numpy as np
import random
import optuna
from sklearn.datasets import fetch_openml
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(int)  # Ensure target labels are integers

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna optimization function for SVM
def objective(trial):
    C = trial.suggest_loguniform('C', 1e-2, 1e2)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    
    model = SVC(C=C, kernel=kernel)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Get best parameters from Optuna
best_params_optuna = study.best_params
best_accuracy_optuna = study.best_value

# Train SVM with Optuna optimized parameters
svm_model_optuna = SVC(C=best_params_optuna["C"], kernel=best_params_optuna["kernel"])
start_time = time.time()
svm_model_optuna.fit(X_train, y_train)
train_time_optuna = time.time() - start_time

y_pred_optuna = svm_model_optuna.predict(X_test)
svm_accuracy_optuna = accuracy_score(y_test, y_pred_optuna)

# Output results
print(f"Optuna Optimized SVM Accuracy: {svm_accuracy_optuna * 100:.2f}%")
print(f"Optuna Optimized SVM Training Time: {train_time_optuna:.2f} seconds")
print(f"Best Parameters Found by Optuna: C={best_params_optuna['C']}, Kernel={best_params_optuna['kernel']}")

## SVM (Bald Eagle)

In [ ]:
import numpy as np
import random
from tensorflow.keras.datasets import mnist
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to [0, 1] range
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten images for SVM
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# Reduce training set to 10,000 samples
subset_size = 10000
x_train, y_train = x_train[:subset_size], y_train[:subset_size]

# BEA-inspired Optimization for SVM
def bea_optimize():
    best_hyperparams = None
    best_acc = 0
    no_improvement_count = 0
    
    population_size = 3
    generations = 2
    
    # Randomly initialize population of hyperparameter sets
    population = [{
        'C': random.choice([0.1, 1, 10, 20]),
        'kernel': random.choice(['linear', 'rbf']),
        'gamma': 'scale'
    } for _ in range(population_size)]
    
    for gen in range(generations):
        print(f"Generation {gen+1}...")
        prev_best_acc = best_acc
        for params in population:
            model = SVC(C=params['C'], kernel=params['kernel'], gamma=params['gamma'])
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)
            acc = accuracy_score(y_test, y_pred)
            
            if acc > best_acc:
                best_acc = acc
                best_hyperparams = params
        
        if best_acc == prev_best_acc:
            no_improvement_count += 1
        else:
            no_improvement_count = 0
        
        if no_improvement_count >= 1:
            print("Early stopping triggered.")
            break
    
    print(f"Best Hyperparameters: {best_hyperparams}")
    print(f"Best Accuracy: {best_acc * 100:.2f}%")
    return best_hyperparams

# Run BEA-inspired optimization
best_params = bea_optimize()

# Train final SVM model with best hyperparameters
final_model = SVC(C=best_params['C'], kernel=best_params['kernel'], gamma=best_params['gamma'])
final_model.fit(x_train, y_train)
y_pred = final_model.predict(x_test)
final_acc = accuracy_score(y_test, y_pred)
print(f"Final Optimized SVM Test Accuracy: {final_acc * 100:.2f}%")

# SVM (Particle Swarm)

In [ ]:
import numpy as np
import random
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist
from sklearn.decomposition import PCA
import time

# Load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape(len(x_train), -1) / 255.0, x_test.reshape(len(x_test), -1) / 255.0

# Reduce dimensionality with PCA (keep 95% variance)
pca = PCA(0.95)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

# PSO Parameters
num_particles = 10  # Number of particles in the swarm
num_iterations = 5  # Number of iterations (generations)
w = 0.7  # Inertia weight
c1 = 1.5  # Cognitive coefficient
c2 = 1.5  # Social coefficient

# Initialize particles
particles = [{'C': random.uniform(0.1, 10), 'gamma': random.choice(['scale', 'auto']), 'kernel': random.choice(['linear', 'rbf'])} for _ in range(num_particles)]
velocities = [{'C': random.uniform(-1, 1)} for _ in range(num_particles)]
best_positions = particles.copy()
best_scores = [float('-inf')] * num_particles

# Global best
global_best_position = None
global_best_score = float('-inf')

for iteration in range(num_iterations):
    print(f"Iteration {iteration + 1}/{num_iterations} started...")
    start_time = time.time()
    
    for i, params in enumerate(particles):
        model = SVC(C=params['C'], kernel=params['kernel'], gamma=params['gamma'])
        model.fit(x_train, y_train)
        acc = accuracy_score(y_test, model.predict(x_test))
        
        if acc > best_scores[i]:
            best_scores[i] = acc
            best_positions[i] = params.copy()
        
        if acc > global_best_score:
            global_best_score = acc
            global_best_position = params.copy()
        
        print(f"  Particle {i + 1}: C={params['C']}, Kernel={params['kernel']}, Gamma={params['gamma']}, Accuracy={acc:.4f}")
    
    # Update velocities and positions
    for i in range(num_particles):
        velocities[i]['C'] = (w * velocities[i]['C'] +
                              c1 * random.random() * (best_positions[i]['C'] - particles[i]['C']) +
                              c2 * random.random() * (global_best_position['C'] - particles[i]['C']))
        
        particles[i]['C'] = max(0.1, particles[i]['C'] + velocities[i]['C'])
    
    print(f"Iteration {iteration + 1} completed in {time.time() - start_time:.2f} seconds")
    print(f"Current Best Score: {global_best_score:.4f}")

print(f"\nFinal Best Hyperparameters: {global_best_position}")
print(f"Final Best Accuracy: {global_best_score * 100:.2f}%")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Iteration 1/5 started...
  Particle 1: C=5.149958702059902, Kernel=linear, Gamma=scale, Accuracy=0.9430
  Particle 2: C=1.2941433556778874, Kernel=rbf, Gamma=scale, Accuracy=0.9840
  Particle 3: C=6.439252660962551, Kernel=linear, Gamma=auto, Accuracy=0.9417
  Particle 4: C=3.6046663238581695, Kernel=rbf, Gamma=auto, Accuracy=0.9794
  Particle 5: C=0.43169650674226934, Kernel=linear, Gamma=scale, Accuracy=0.9457
  Particle 6: C=7.299701051771945, Kernel=linear, Gamma=auto, Accuracy=0.9420
  Particle 7: C=2.1176633082563137, Kernel=linear, Gamma=scale, Accuracy=0.9446
  Particle 8: C=5.325918895883778, Kernel=rbf, Gamma=auto, Accuracy=0.9802
  Particle 9: C=3.1578531346331795, Kernel=linear, Gamma=scale, Accuracy=0.9439
  Particle 10: C=8.2332568509797, Kernel=rbf, Gamma=auto, Accuracy=0.9819
Iteration 1 completed in 1816.67 seconds
Current Best Score: 0.9840
Iteration 2/5 started...
  Particle 1: C=1.205413267273821, Kernel=linear, Gam

## 3 layered CNN

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import numpy as np

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to [0, 1] range
x_train, x_test = x_train / 255.0, x_test / 255.0

# Expand dimensions to match CNN input shape
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Define CNN model
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Evaluate model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 17ms/step - accuracy: 0.9190 - loss: 0.2925 - val_accuracy: 0.9876 - val_loss: 0.0367
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.9857 - loss: 0.0436 - val_accuracy: 0.9843 - val_loss: 0.0492
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.9903 - loss: 0.0280 - val_accuracy: 0.9864 - val_loss: 0.0403
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.9943 - loss: 0.0175 - val_accuracy: 0.9924 - val_loss: 0.0231
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.9952 - loss: 0.0139 - val_accuracy: 0.9912 - val_loss: 0.0242
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 16ms/step - accuracy: 0.9961 - loss: 0.0105 - val_accuracy: 0.9911 - val_loss: 0.0279
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 17ms/step - accuracy: 0.9973 - loss: 0.0091 - val_accuracy: 0.9894 - val_loss: 0.0362
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.9970 -